In [1]:
!pip freeze | grep apache-beam==2.10.0 || pip install apache-beam[gcp]

    100% |████████████████████████████████| 2.5MB 566kB/s 
    100% |████████████████████████████████| 51kB 12.4MB/s 
    100% |████████████████████████████████| 276kB 5.1MB/s 
    100% |████████████████████████████████| 1.2MB 1.2MB/s 
    100% |████████████████████████████████| 512kB 3.1MB/s 
    100% |████████████████████████████████| 81kB 11.0MB/s 
    100% |████████████████████████████████| 133kB 10.0MB/s 
    100% |████████████████████████████████| 11.6MB 121kB/s 
    100% |████████████████████████████████| 153kB 10.1MB/s 
    100% |████████████████████████████████| 1.0MB 1.4MB/s 
    100% |████████████████████████████████| 61kB 13.7MB/s 
    100% |████████████████████████████████| 10.7MB 131kB/s 
    100% |████████████████████████████████| 829kB 1.8MB/s 
    100% |████████████████████████████████| 225kB 6.4MB/s 
    100% |████████████████████████████████| 92kB 8.8MB/s 
    100% |████████████████████████████████| 337kB 4.2MB/s 
    100% |████████████████████████████████| 92kB 14.1

In [2]:
!pip freeze | grep tensorflow==1.12.0 || pip install tensorflow==1.12.0

    100% |████████████████████████████████| 83.1MB 15kB/s 
    100% |████████████████████████████████| 61kB 14.7MB/s 
  Using cached https://files.pythonhosted.org/packages/e6/35/f187bdf23be87092bd0f1200d43d23076cee4d0dec109f195173fd3ebc79/mock-2.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b8/be/3bb6d8241b5ed1f8437169df53e7dd6ca986174e022585de15087a848c99/grpcio-1.19.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/c4/33/8ec8dcdb4ede5d453047bbdbd01916dbaccdb63e98bba60989718f5f0876/numpy-1.16.2-cp27-cp27mu-manylinux1_x86_64.whl
    100% |████████████████████████████████| 3.1MB 518kB/s 
    100% |████████████████████████████████| 102kB 11.7MB/s 
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 61kB 10.6MB/s 
  Using cached https://files.pythonhosted.org/packages/ea

In [3]:
!pip freeze > requirements.txt

In [4]:
import apache_beam as beam
import datetime
import os

ImportError: No module named 'apache_beam'

In [ ]:
PROJECT = 'qwiklabs-gcp-3f19cbba7aa3ae63'
os.environ['PROJECT'] = PROJECT
os.environ['CLOUDSDK_PYTHON'] = 'python3'

In [ ]:
%%bash
gcloud config set project $PROJECT

In [ ]:
query = """
#standardSQL
SELECT DISTINCT product_id, image_url FROM `qwiklabs-gcp-3f19cbba7aa3ae63.project.raw`
"""

In [ ]:
# This will take the image url and product id, and fetch the actual image
class TransformImages(beam.DoFn):
  def process(self, element):
    import requests
    from PIL import Image
    import logging
    import io
    
    uri = element['image_url']
    id = element['product_id']
    
    # some urls are missing the "http:" part
    if uri[:2] == '//':
        uri = 'http:' + uri
    
    try:
      res = requests.get(url=uri)
      image_bytes = res.content
      img = Image.open(io.BytesIO(image_bytes)).convert('RGB')
    
      # if you want to resize the image, uncomment the next line
      #img = img.resize((250, 250), Image.ANTIALIAS)
    except Exception as e:
      logging.exception('Error processing image %s: %s', uri, str(e))
      return

    output = io.BytesIO()
    img.save(output, 'jpeg')
    image_bytes = output.getvalue()
    yield id, image_bytes
    
# This will write the image bytes to GCS
class WriteToStorage(beam.DoFn):
  def process(self, element):
    from apache_beam.io import filesystems
    (name, image_bytes) = element
    path = 'gs://project-sample/dataset1/{}.jpeg'.format(name)
    writer = filesystems.FileSystems.create(path)
    writer.write(image_bytes)
    writer.close()


def preprocess(runner):
  job_name = 'test-preprocess-images' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')
  print('Launching Dataflow job {} ... hang on'.format(job_name))

  options = {
    'staging_location': 'gs://project-sample/out/tmp/staging',
    'temp_location': 'gs://project-sample/out/tmp',
    'job_name': job_name,
    'requirements_file': 'requirements.txt',
    'project': PROJECT,
    'runner': runner,
  }
  
  #instantiate PipelineOptions object using options dictionary
  opts = beam.pipeline.PipelineOptions(flags=[], **options)

  #instantantiate Pipeline object using PipelineOptions
  with beam.Pipeline(options=opts) as p:
    (
      p | 'read' >> beam.io.Read(beam.io.BigQuerySource(query=query, use_standard_sql=True))
        | 'transform' >> beam.ParDo(TransformImages())
        | 'write' >> beam.ParDo(WriteToStorage())
    )
  print("Done")